## Analyzing borrowers’ risk of defaulting

This project is to prepare a report for a bank’s loan division. We will find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer 
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Wesley\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wesley\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Step 1. Open the data file and have a look at the general information. 

In [2]:
credit_data = pd.read_csv("credit_scoring_eng.csv")
credit_data.info()
#we can increment the value passed to the head method if there aren't any obvious issues.
print(credit_data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024

### Conclusion

From our initial observation of the data utilizing panda's info() method, we can see that two columns have less non-null values than the rest - 'days_employed', and 'total_income', with values 19351, compared to 21525. The ~2000 null values in each column will need to be filled.

Let's now take a look at the first few rows to determine if there are any glaringly obvious issues. We'll look at each column to determine what else needs to be fixed, and if there are any other initial observations:

**Children**: no obvious issues.
**Days employed**: in addition to null values, there are negative values. These need to be converted to positive values. There are also unrealistic values for data such as 340266.072047 and 400281.136913. These also need to be changed. Type can also be converted from float to int, since we're just counting days.
**DOB years**: no obvious issues.
**Education**: strings aren't normalized. We need to normalize string values.
**Education ID**: Education corresponds to an ID. We can use this ID to analyze data rather than using the education column, to prevent errors from potentially malformed strings. No obvious issues.
**Family status/Family ID**: Family status is similar to education, a categorical variable corresponding to an ID. No obvious issues with these rows.
**Gender**: Categorical binary variable. The only issue is people might have an issue with a gender binary. Maybe we can add more genders to the category. :) otherwise, no obvious issues.
**Income Type**: no obvious issues.
**Debt**: binary with 0 representing no defaults on loans, and 1 representing past default(s) on loan(s). No obivous issues.
**Total Income**: null values
**Purpose**: random string inputted by user. This needs to be lemmatized.


Based on our initial take of our data, these following columns need to be fixed, or worked on before we can analyze the data:
**Days employed
Education
Total income
Purpose**

Let's start with filling in missing values in **'days_employed'** and **'total_income'**. Since these are both quantitative variables, we'll fill the null values with the mean value of the column for now. 

The 'days_employed' column need negative values to be changed to positive, and unrealistic values to be changed to realistic values before we can accurately determine the mean.

In the future, if we want to use these columns for data analysis such as finding a correlation between days of employment and total income, we may need to ask the developers to find the missing values to more accurately determine the data.


### Step 2. Data preprocessing

### Processing missing values

In [3]:
#get mean of income
income_avg = credit_data['total_income'].mean()
#firstly check the income_avg to ensure it's realistic, or it makes sense.
print("Income average: ", income_avg)

#then fill the null values with the mean
credit_data['total_income'].fillna(value = income_avg, inplace = True)

#convert negatives to positives
credit_data['days_employed'] = credit_data['days_employed'].abs()

#remove unrealistic data in days_employed, so our mean is realistic. Let's remove anything > 29,200(365 days * 80 years).
credit_data.loc[credit_data['days_employed']>29200, 'days_employed'] = None

#get mean of days employed
employed_avg = credit_data['days_employed'].mean()

#now check the mean to see if it's sensible as well
print("Days employed average: ", employed_avg)
#fill null values with mean - this includes the previously unrealistic data.
credit_data['days_employed'].fillna(value = employed_avg, inplace=True)

#lastly, normalize strings in the education column.
credit_data['education'] = credit_data['education'].str.lower()

#now check the data. Looks fine now.
print('\n---------------------------------------------\n')
print(credit_data.head(10))

Income average:  26787.56835465871
Days employed average:  2353.015931998879

---------------------------------------------

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    bachelor's degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0    2353.015932         53  secondary education             1   
5         0     926.185831         27    bachelor's degree             0   
6         0    2879.202052         43    bachelor's degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35    bachelor's degree             0   
9         0    2188.756445         41  secondary education             1   

       family_status  family_status_id

### Conclusion

We've now fixed our days employed and total income columns. Null values, negative values, and unrealistic numbers were removed. But how did we get these values in the first place?

**Null values**: User could have abstained from entering the information, if it wasn't mandated.
**Negative values**: Developer may have saved the data as a negative value, and this made its way into the data without being reverted back to a positive number. Days employed should never be a negative value in the first place.
**Unrealistic numbers**: Data was corrupted somewhere, or users falsely inputted these numbers.

### Data type replacement

In [4]:
#convert days employed from float to int
credit_data['days_employed'] = credit_data['days_employed'].astype('int')

#check data
credit_data.info()
print(credit_data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int32  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB
   children  days_employed  dob_years            education  education_id  \
0         1           8437         42    bachelor's degree             0   
1        

### Conclusion

Here we convert days employed from a float to an int. it isn't entirely necessary, but we can't really have a fraction of a day of employment so we might as well change it. Now all the data types seem to be correct. We use the astype() method from pandas because to_numeric() produces a float, and it's already a float in the first place.

### Processing duplicates

In [5]:
credit_data = credit_data.drop_duplicates()

credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  int32  
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

Seems like there were a few duplicate rows. Entries reduced from 21525 --> 21471. We use pandas' drop_duplicates() method, because why would we manually scan for duplicate rows?

We then need to figure out why there were duplicates - users may have inputted their data several times, and/or data was replicated somewehere and added into the database. Either way there shouldn't be duplicates in the first place... *or at least, not this many.*

### Categorizing Data

In [6]:
'''
we first need to define a function for classifying income level. Since we have our median income, 26787.56835465867,
we can distribute income levels between 'low', 'middle', and 'high' income. Let's do something like

low < 18000 < median < 36000 < high

since the median is ~ 27000.

we don't need a try/except block since 'income' is an integer that falls underneath one of these three conditional statements.
'''
def income_level(income):
    if income < 18000:
        return 'low'
    elif income >= 18000 and income <=36000:
        return 'med'
    else:
        return 'high'
    
#and now we'll apply this function to a new column.
credit_data['income_level'] = credit_data['total_income'].apply(income_level)

#and let's see our result.
print(credit_data['income_level'].value_counts())

med     11691
low      5990
high     3773
Name: income_level, dtype: int64


In [7]:
'''
now we need to categorize purpose.

let's list potential purposes:
education
wedding
house
property
real estate
university

now we'll categorize them:

education, university ==> education
house, property, real estate ==> house
wedding ==> wedding

we'll create a function that lemmatizes words, and we'll apply this function to a new column.

any 'purpose' that doesn't fall under any of these categories will be caught in the except block.
'''

def lemmatized(purpose):
    try:
        lemmatizer = WordNetLemmatizer()
        words = nltk.word_tokenize(purpose)
        lemmas_v = [lemmatizer.lemmatize(w, pos = 'v') for w in words]
        house = ['house','property','estate']
        education = ['university','education', 'educate']
        car = ['car','cars']
        wedding = ['wed','wedding']
        for word in lemmas_v:
            if word in house:
                return 'housing'
            elif word in education:
                return 'education'
            elif word in car:
                return 'car'
            elif word in wedding:
                return 'wedding'

        lemmas_n = [lemmatizer.lemmatize(w, pos = 'v') for w in words]

        for word in lemmas_n:
            if word in house:
                return 'housing'
            elif word in education:
                return 'education'
            elif word in car:
                return 'car'
            elif word in wedding:
                return 'wedding'
            
        raise Exception()
    except:
        print("purpose: ", purpose,"does not fall into any classified category")

#apply it
credit_data['actual_purpose'] = credit_data['purpose'].apply(lemmatized)

#our result
print(credit_data['actual_purpose'].value_counts())



housing      10811
car           4306
education     4013
wedding       2324
Name: actual_purpose, dtype: int64


### Conclusion

The questions we need to answer are:

1. Is there a relation between having kids and repaying a loan on time?
2. Is there a relation between marital status and repaying a loan on time?
3. Is there a relation between income level and repaying a loan on time?
4. How do different loan purposes affect on-time repayment of the loan?

So we need categories for:
**loans
having kids
marital status
income level
purpose**

The only non-categorical data columns are income level and purpose, so we need to categorize both to answer our relational question. Without categorization is hard to draw meaningful conclusions from our data.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [8]:
data = credit_data.groupby(['children','debt']).size().reset_index(name='counts')

table = data.pivot(index='children', columns='debt', values='counts')

table.columns = ['no debt', 'debt']
table['percentage'] = table['no debt']/(table['no debt']+table['debt'])
table['percentage'] = table['percentage'].astype(float).map("{:.2%}".format)
table

,no debt,debt,percentage
children,,,
-1,46.0,1.0,97.87%
0,13028.0,1063.0,92.46%
1,4364.0,444.0,90.77%
2,1858.0,194.0,90.55%
3,303.0,27.0,91.82%
4,37.0,4.0,90.24%
5,9.0,NaN,nan%
20,68.0,8.0,89.47%


### Conclusion

The probability of debt repayment is as follows, based on number of children:

**0 children: 92.46%
1 child: 90.77%
2 children: 90.55%
3 children: 91.82%
4 children: 90.24%
5 children: 100%**

Based on our collected data, we can assume that there isn't a significant statistical correlation between having children and repaying a loan on time.

We also have some interesting data where the we have several rows with -1 child, and 20 children! Some questions to ask:
What does -1 mean? No children? 1 child? an undefined number of children?
**how are there 76 families with 20 children????**

- Is there a relation between marital status and repaying a loan on time?

In [9]:
data = credit_data.groupby(['family_status','debt']).size().reset_index(name='counts')

table = data.pivot(index='family_status', columns='debt', values='counts')

table.columns = ['no debt', 'debt']
table['percentage'] = table['no debt']/(table['no debt']+table['debt'])
table['percentage'] = table['percentage'].astype(float).map("{:.2%}".format)
table

,no debt,debt,percentage
family_status,,,
civil partnership,3763,388,90.65%
divorced,1110,85,92.89%
married,11408,931,92.45%
unmarried,2536,274,90.25%
widow / widower,896,63,93.43%


### Conclusion

The probability of debt repayment is as follows, based on number of family status:

**civil partnership: 90.65%
divorced: 92.89%
married: 92.44%
unmarried: 90.25%
widow/widower: 93.43%**

Based on our data, there isn't a significant statistical correlation between debt repayment and family status.

- Is there a relation between income level and repaying a loan on time?

In [10]:
data = credit_data.groupby(['income_level','debt']).size().reset_index(name='counts')

table = data.pivot(index='income_level', columns='debt', values='counts')

table.columns = ['no debt', 'debt']
table['percentage'] = table['no debt']/(table['no debt']+table['debt'])
table['percentage'] = table['percentage'].astype(float).map("{:.2%}".format)
table

,no debt,debt,percentage
income_level,,,
high,3506,267,92.92%
low,5512,478,92.02%
med,10695,996,91.48%


### Conclusion

The probability of debt repayment is as follows, based on income level:

**high: 92.92%
low: 92.02%
med: 91.48**

Based on our data, there isn't a significant statistical correlation between debt repayment and income level.

- How do different loan purposes affect on-time repayment of the loan?

In [11]:
data = credit_data.groupby(['actual_purpose','debt']).size().reset_index(name='counts')

table = data.pivot(index='actual_purpose', columns='debt', values='counts')

table.columns = ['no debt', 'debt']
table['percentage'] = table['no debt']/(table['no debt']+table['debt'])
table['percentage'] = table['percentage'].astype(float).map("{:.2%}".format)
table

,no debt,debt,percentage
actual_purpose,,,
car,3903,403,90.64%
education,3643,370,90.78%
housing,10029,782,92.77%
wedding,2138,186,92.00%


### Conclusion

The probability of debt repayment is as follows, based on purpose:

**car: 90.64%
education: 90.78%
housing: 92.77%
wedding: 92.00%**

Based on our data, there isn't a significant statistical correlation between debt repayment and income level.

### Step 4. General conclusion

Based on the analysis of the data, we can conclude that there aren't significant differences between the categories that we analyzed and debt repayment. Typical percentage of debt repayment across all boards range from ~90-92%. 